In [8]:
import math  # Importer le module math pour utiliser les fonctions mathématiques

# taux sans risque (du livre blanc, p5)
rates = [0.000305, 0.000286]  # Taux sans risque pour les échéances proches et suivantes

# fichiers de données d'options SPX utilisés pour le calcul du VIX (livre blanc, appendice A)
datafiles = ['./vixnearterm.dat', './vixnextterm.dat']  # Chemins des fichiers de données d'options SPX

# niveau de verbosité : 0 pour juste le résultat, 1 pour quelques sorties, 2 pour plus de sorties
verbose = 0  # Niveau de verbosité pour les sorties du script

# obtenir les données d'options à partir de vixnearterm.dat et vixnextterm.dat pour les options expirant aux dates d'échéance proche et suivante
quotedata = [[], []]  # Initialiser une liste vide pour stocker les données des options
for j in (0, 1):  # Boucle pour lire les deux fichiers de données
    f = open(datafiles[j])  # Ouvrir le fichier
    for line in f:  # Lire chaque ligne du fichier
        ar = []  # Initialiser une liste pour stocker les valeurs d'une ligne
        for v in line.strip().split("\t"):  # Diviser la ligne par tabulation
            ar.append(float(v))  # Convertir chaque valeur en float et l'ajouter à la liste
        quotedata[j].append(ar)  # Ajouter la ligne de données à la liste quotedata
    f.close()  # Fermer le fichier

# obtenir le nombre de minutes et d'années jusqu'aux dates d'échéance proche et suivante (p5)
Nt = [854 + 510 + 34560, 854 + 900 + 44640]  # Nombre de minutes jusqu'aux dates d'échéance proche et suivante
T = [Nt[0] / (60 * 24 * 365), Nt[1] / (60 * 24 * 365)]  # Conversion des minutes en années

if verbose >= 1:  # Si le niveau de verbosité est supérieur ou égal à 1
    print('Nt:', Nt)  # Afficher Nt
    print('T:', T)  # Afficher T

# Étape 1 : Sélectionner les options à utiliser dans le calcul du VIX
# Calculer F pour les échéances proche et suivante (p6)
F = [None, None]  # Initialiser la liste F pour les échéances proche et suivante
for j in (0, 1):  # Boucle pour les deux échéances
    mindiff = None  # Initialiser la différence minimale
    diff = None  # Initialiser la différence actuelle
    Fstrike = None  # Initialiser le prix d'exercice F
    Fcall = None  # Initialiser le prix moyen de l'option call F
    Fput = None  # Initialiser le prix moyen de l'option put F
    for d in quotedata[j]:  # Boucle pour chaque ligne de données
        diff = abs(((d[1] + d[2]) / 2) - ((d[3] + d[4]) / 2))  # Calculer la différence entre les prix moyens des calls et puts
        if mindiff is None or diff < mindiff:  # Si la différence actuelle est la plus petite
            mindiff = diff  # Mettre à jour la différence minimale
            Fstrike = d[0]  # Mettre à jour le prix d'exercice F
            Fcall = (d[1] + d[2]) / 2  # Mettre à jour le prix moyen de l'option call F
            Fput = (d[3] + d[4]) / 2  # Mettre à jour le prix moyen de l'option put F
    F[j] = Fstrike + math.exp(rates[j] * T[j]) * (Fcall - Fput)  # Calculer F pour l'échéance actuelle

if verbose >= 1:  # Si le niveau de verbosité est supérieur ou égal à 1
    print('F:', F)  # Afficher F

# sélectionner les options à utiliser dans le calcul du VIX (p6,7)
selectedoptions = [[], []]  # Initialiser la liste des options sélectionnées
k0 = [None, None]  # Initialiser k0 pour les échéances proche et suivante
for j in (0, 1):  # Boucle pour les deux échéances
    i = 0  # Initialiser l'index
    for d in quotedata[j]:  # Boucle pour chaque ligne de données
        if d[0] < F[j]:  # Si le prix d'exercice est inférieur à F
            k0[j] = d[0]  # Mettre à jour k0
            k0i = i  # Mettre à jour l'index k0i
        i += 1  # Incrémenter l'index

    d = quotedata[j][k0i]  # Obtenir les données pour k0
    ar = [d[0], 'put/call average', (((d[1] + d[2]) / 2) + ((d[3] + d[4]) / 2)) / 2]  # Calculer le prix moyen put/call
    selectedoptions[j].append(ar)  # Ajouter à la liste des options sélectionnées

    i = k0i - 1  # Décrémenter l'index pour commencer avant k0
    b = True  # Initialiser le drapeau de boucle
    previousbid = None  # Initialiser le prix de l'offre précédente
    while b and i >= 0:  # Boucle jusqu'à atteindre le début de la liste ou une condition de fin
        d = quotedata[j][i]  # Obtenir les données actuelles
        if d[3] > 0:  # Si le prix de l'offre put est positif
            ar = [d[0], 'put', (d[3] + d[4]) / 2]  # Calculer le prix moyen put
            selectedoptions[j].insert(0, ar)  # Insérer à la position 0
        else:
            if previousbid == 0:  # Si l'offre précédente était zéro
                b = False  # Mettre fin à la boucle
        previousbid = d[3]  # Mettre à jour l'offre précédente
        i -= 1  # Décrémenter l'index

    i = k0i + 1  # Incrémenter l'index pour commencer après k0
    b = True  # Initialiser le drapeau de boucle
    previousbid = None  # Initialiser le prix de l'offre précédente
    while b and i < len(quotedata[j]):  # Boucle jusqu'à atteindre la fin de la liste ou une condition de fin
        d = quotedata[j][i]  # Obtenir les données actuelles
        if d[1] > 0:  # Si le prix de l'offre call est positif
            ar = [d[0], 'call', (d[1] + d[2]) / 2]  # Calculer le prix moyen call
            selectedoptions[j].append(ar)  # Ajouter à la liste des options sélectionnées
        else:
            if previousbid == 0:  # Si l'offre précédente était zéro
                b = False  # Mettre fin à la boucle
        previousbid = d[1]  # Mettre à jour l'offre précédente
        i += 1  # Incrémenter l'index

if verbose == 2:  # Si le niveau de verbosité est égal à 2
    print('selectedoptions (near term):')  # Afficher les options sélectionnées pour l'échéance proche
    for e in selectedoptions[0]:  # Boucle pour chaque option sélectionnée
        print('', e)  # Afficher l'option
    print('selectedoptions (next term):')  # Afficher les options sélectionnées pour l'échéance suivante
    for e in selectedoptions[1]:  # Boucle pour chaque option sélectionnée
        print('', e)  # Afficher l'option

# Étape 2 : Calculer la volatilité pour les options à court terme et à long terme (p8)
for j in (0, 1):  # Boucle pour les deux échéances
    i = 0  # Initialiser l'index
    for d in selectedoptions[j]:  # Boucle pour chaque option sélectionnée
        if i == 0:  # Si c'est la première option
            deltak = selectedoptions[j][1][0] - selectedoptions[j][0][0]  # Calculer delta k
        elif i == len(selectedoptions[j]) - 1:  # Si c'est la dernière option
            deltak = selectedoptions[j][i][0] - selectedoptions[j][i - 1][0]  # Calculer delta k
        else:
            deltak = (selectedoptions[j][i + 1][0] - selectedoptions[j][i - 1][0]) / 2  # Calculer delta k pour une option intermédiaire
        contributionbystrike = (deltak / (d[0] * d[0])) * math.exp(rates[j] * T[j]) * d[2]  # Calculer la contribution par strike
        selectedoptions[j][i].append(contributionbystrike)  # Ajouter la contribution à la liste des options sélectionnées
        i += 1  # Incrémenter l'index

if verbose == 2:  # Si le niveau de verbosité est égal à 2
    print('contributions by strike (near term):')  # Afficher les contributions par strike pour l'échéance proche
    for e in selectedoptions[0]:  # Boucle pour chaque option sélectionnée
        print('', e)  # Afficher l'option
    print('contributions by strike (next term):')  # Afficher les contributions par strike pour l'échéance suivante
    for e in selectedoptions[1]:  # Boucle pour chaque option sélectionnée
        print('', e)  # Afficher l'option

# Calculer la contribution agrégée par strike pour les échéances proche et suivante
aggregatedcontributionbystrike = [None, None]  # Initialiser la liste pour les contributions agrégées
for j in (0, 1):  # Boucle pour les deux échéances
    aggregatedcontributionbystrike[j] = 0  # Initialiser la contribution agrégée à zéro
    for d in selectedoptions[j]:  # Boucle pour chaque option sélectionnée
        aggregatedcontributionbystrike[j] += d[3]  # Ajouter la contribution de chaque option
    aggregatedcontributionbystrike[j] = (2 / T[j]) * aggregatedcontributionbystrike[j]  # Calculer la contribution agrégée finale

# Calculer sigma^2 pour les échéances proche et suivante
sigmasquared = [None, None]  # Initialiser la liste sigma^2
for j in (0, 1):  # Boucle pour les deux échéances
    sigmasquared[j] = aggregatedcontributionbystrike[j] - (1 / T[j]) * (F[j] / k0[j] - 1) * (F[j] / k0[j] - 1)  # Calculer sigma^2

if verbose:  # Si le niveau de verbosité est supérieur à 0
    print('sigmasquared:', sigmasquared)  # Afficher sigma^2

# Étape 3 : Calculer la moyenne pondérée sur 30 jours de sigmasquared[0] et sigmasquared[1] (p9)
N30 = 30 * 1440  # Nombre de minutes dans 30 jours
N365 = 365 * 1440  # Nombre de minutes dans une année
VIX = 100 * math.sqrt(((T[0] * sigmasquared[0]) * (Nt[1] - N30) / (Nt[1] - Nt[0]) + (T[1] * sigmasquared[1]) * (N30 - Nt[0]) / (Nt[1] - Nt[0])) * N365 / N30)  # Calculer le VIX

print('VIX:', VIX)  # Afficher le VIX


VIX: 13.68582053794788
